# Worldwide Earthquake Events API - Silver Layer Processing

In [ ]:
from pyspark.sql.functions import col
from pyspark.sql.types import TimestampType

In [ ]:
# df est maintenant un DataFrame Spark contenant les données JSON
df = spark.read.option("multiline", "true").json(f"Files/{start_date}_earthquake_data.json")

In [ ]:
# Restructurer les données sismiques en extrayant et renommant les attributs clés pour faciliter l'analyse
df = \
df.\
    select(
        'id',
        col('geometry.coordinates').getItem(0).alias('longitude'),
        col('geometry.coordinates').getItem(1).alias('latitude'),
        col('geometry.coordinates').getItem(2).alias('elevation'),
        col('properties.title').alias('title'),
        col('properties.place').alias('place_description'),
        col('properties.sig').alias('sig'),
        col('properties.mag').alias('mag'),
        col('properties.magType').alias('magType'),
        col('properties.time').alias('time'),
        col('properties.updated').alias('updated')
        )

In [ ]:
# Convertir les colonnes 'time' et 'updated' de millisecondes vers un format timestamp
df = df.\
    withColumn('time', col('time')/1000).\
    withColumn('updated', col('updated')/1000).\
    withColumn('time', col('time').cast(TimestampType())).\
    withColumn('updated', col('updated').cast(TimestampType()))

In [ ]:
# Ajouter les données à la table silver
df.write.mode('append').saveAsTable('earthquake_events_silver')